## Implementación Práctica: Transformer desde Cero

En este cuaderno, implementaremos los componentes clave de un modelo Transformer siguiendo la arquitectura original "Attention Is All You Need". Ejecuta cada celda de código para definir las clases y funciones necesarias.

### Importaciones y Clases Auxiliares
Importamos las librerías necesarias y definimos algunas clases fundamentales como la Normalización de Capa (LayerNorm) y la Conexión Residual (SublayerConnection).

In [ ]:
import torch
import torch.nn as nn
import math
import copy # Para clonar módulos

class LayerNorm(nn.Module):
    # Construye una capa de LayerNorm.
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        # Parámetros aprendibles gamma (a_2) y beta (b_2)
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps # Pequeño valor para evitar división por cero

    def forward(self, x):
        # x shape: (batch_size, seq_len, features)
        # Calcula la media y desviación estándar sobre la última dimensión (features)
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        # Aplica la normalización
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    # Conexión residual seguida de LayerNorm. Nota: La normalización va ANTES de la subcapa aquí.
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        # Aplica la conexión residual a cualquier subcapa del mismo tamaño.
        # Normaliza x, pasa por la subcapa (con dropout), y añade la entrada original x (conexión residual)
        return x + self.dropout(sublayer(self.norm(x)))

print("Clases LayerNorm y SublayerConnection definidas.")


Clases LayerNorm y SublayerConnection definidas.


### Atención Multi-Cabeza (Multi-Head Attention)

In [ ]:
class MultiHeadAttention(nn.Module):
    # Implementación de la atención multi-cabeza.
    def __init__(self, h, d_model, dropout=0.1):
        # Toma el número de cabezas (h) y la dimensión del modelo (d_model).
        super(MultiHeadAttention, self).__init__()
        assert d_model % h == 0
        # Asumimos que d_v siempre es igual a d_k
        self.d_k = d_model // h
        self.h = h
        # Creamos 4 capas lineales: para Q, K, V y la salida final
        self.linears = nn.ModuleList([copy.deepcopy(nn.Linear(d_model, d_model)) for _ in range(4)])
        self.attn = None # Para almacenar los pesos de atención para visualización/análisis
        self.dropout = nn.Dropout(p=dropout)

    def attention(self, query, key, value, mask=None, dropout=None):
        # Calcula la salida de la atención escalada por producto punto.
        d_k = query.size(-1)
        # 1. Calcula scores: (Batch, h, Seq_q, d_k) @ (Batch, h, d_k, Seq_k) -> (Batch, h, Seq_q, Seq_k)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        # 2. Aplica máscara (si existe)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9) # Rellena con valor muy negativo donde mask es 0
        # 3. Aplica softmax para obtener pesos
        p_attn = torch.softmax(scores, dim=-1)
        if dropout is not None:
            p_attn = dropout(p_attn)
        # 4. Multiplica pesos por Values: (Batch, h, Seq_q, Seq_k) @ (Batch, h, Seq_k, d_k) -> (Batch, h, Seq_q, d_k)
        return torch.matmul(p_attn, value), p_attn

    def forward(self, query, key, value, mask=None):
        # Implementa la figura 2 del paper.
        if mask is not None:
            # Aplica la misma máscara a todas las cabezas
            mask = mask.unsqueeze(1) # (Batch, 1, Seq_q, Seq_k) o (Batch, 1, 1, Seq_k)
        nbatches = query.size(0)

        # 1) Hacer proyección lineal en batch de d_model => h x d_k
        # query, key, value: (Batch, Seq, d_model)
        # -> aplica linear y view -> (Batch, Seq, h, d_k)
        # -> transpose -> (Batch, h, Seq, d_k)
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Aplicar atención en todos los vectores proyectados en batch.
        # x: (Batch, h, Seq_q, d_k)
        # self.attn: (Batch, h, Seq_q, Seq_k)
        x, self.attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concatenar" usando view y aplicar proyección lineal final.
        # x: (Batch, h, Seq_q, d_k) -> transpose -> (Batch, Seq_q, h, d_k)
        # -> contiguous + view -> (Batch, Seq_q, d_model)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        # Aplica la capa lineal final W_o
        return self.linears[-1](x)

print("Clase MultiHeadAttention definida.")


Clase MultiHeadAttention definida.


###Red Feed-Forward (Position-wise)
Cada capa del Transformer también contiene una red feed-forward simple que se aplica independientemente a cada posición.

In [ ]:
class PositionwiseFeedForward(nn.Module):
    # Implementa una red FFN.
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff) # Capa de expansión
        self.w_2 = nn.Linear(d_ff, d_model) # Capa de contracción
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU() # Activación ReLU (común)

    def forward(self, x):
        # x: (Batch, Seq, d_model)
        # -> w_1 -> (Batch, Seq, d_ff)
        # -> activation -> (Batch, Seq, d_ff)
        # -> dropout -> (Batch, Seq, d_ff)
        # -> w_2 -> (Batch, Seq, d_model)
        return self.w_2(self.dropout(self.activation(self.w_1(x))))

print("Clase PositionwiseFeedForward definida.")


Clase PositionwiseFeedForward definida.


### Embeddings y Codificación Posicional
Convertimos los tokens de entrada en vectores (embeddings) y añadimos información sobre su posición en la secuencia.

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        # vocab: tamaño del vocabulario.
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model) # Capa de embedding
        self.d_model = d_model

    def forward(self, x):
        # x: (Batch, Seq) - Índices de tokens
        # -> lut -> (Batch, Seq, d_model)
        # Multiplica por sqrt(d_model) según el paper
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    # Implementa la codificación posicional PE.
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        # max_len: longitud máxima de secuencia esperada.
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Calcula los valores de codificación posicional una vez en log space
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # (max_len, 1)
        # Término de división para las frecuencias
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Calcula seno para posiciones pares y coseno para impares
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # Añadir dimensión de batch: (1, max_len, d_model)
        # Registrar 'pe' como buffer, no como parámetro entrenable
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (Batch, Seq, d_model)
        # Añade la codificación posicional a los embeddings
        # Necesitamos tomar solo las primeras 'Seq' codificaciones posicionales
        # self.pe[:, :x.size(1)] -> (1, Seq, d_model)
        x = x + self.pe[:, :x.size(1)].requires_grad_(False) # No requiere gradiente
        return self.dropout(x)

print("Clases Embeddings y PositionalEncoding definidas.")


Clases Embeddings y PositionalEncoding definidas.


### Capas del Codificador y Decodificador
Ahora ensamblamos las subcapas (atención y feed-forward) para formar las capas del codificador y decodificador.

In [ ]:
class EncoderLayer(nn.Module):
    # El Encoder se compone de auto-atención y feed-forward.
    def __init__(self, size, self_attn, feed_forward, dropout):
        # size: d_model.
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn # Subcapa de auto-atención
        self.feed_forward = feed_forward # Subcapa feed-forward
        # Dos conexiones residuales + LayerNorm
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(2)])
        self.size = size

    def forward(self, x, mask):
        # Sigue la conexión definida en SublayerConnection.
        # 1. Auto-atención (Query, Key, Value son iguales a x)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # 2. Feed-forward
        return self.sublayer[1](x, self.feed_forward)

class DecoderLayer(nn.Module):
    # El Decoder se compone de auto-atención, atención fuente-objetivo y feed-forward.
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn # Auto-atención (enmascarada) sobre la salida del decoder
        self.src_attn = src_attn   # Atención sobre la salida del encoder (memoria)
        self.feed_forward = feed_forward
        # Tres conexiones residuales + LayerNorm
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(3)])

    def forward(self, x, memory, src_mask, tgt_mask):
        # x: entrada del decoder (salida previa + pos encoding)
        # memory: salida del encoder
        # src_mask: máscara para el padding en la secuencia de entrada (encoder)
        # tgt_mask: máscara para el padding y para evitar atención futura en la secuencia de salida (decoder)

        m = memory
        # 1. Auto-atención enmascarada (Query, Key, Value son iguales a x)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # 2. Atención sobre la salida del encoder (Query=x, Key=memory, Value=memory)
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        # 3. Capa FeedForward
        return self.sublayer[2](x, self.feed_forward)

print("Clases EncoderLayer y DecoderLayer definidas.")


Clases EncoderLayer y DecoderLayer definidas.


### Ensamblaje del Codificador y Decodificador
Apilamos múltiples capas para formar el codificador y el decodificador completos.

In [ ]:
class Encoder(nn.Module):
    # Encoder central con N capas.
    def __init__(self, layer, N):
        # layer: una instancia de EncoderLayer. N: número de capas.
        super(Encoder, self).__init__()
        # Clona la capa N veces
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        # Normalización final después de la última capa
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        # Pasa la entrada (y máscara) a través de cada capa.
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class Decoder(nn.Module):
    # Decoder genérico con N capas.
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        # Pasa la entrada (y máscaras) a través de cada capa.
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

print("Clases Encoder y Decoder definidas.")


Clases Encoder y Decoder definidas.


### Modelo Transformer Completo y Generador
Finalmente, unimos el codificador, el decodificador y las capas de embedding, junto con una capa final (Generador) para producir las probabilidades de salida sobre el vocabulario.

In [ ]:
class Generator(nn.Module):
    # Define la capa de generación lineal estándar + softmax.
    def __init__(self, d_model, vocab):
        # vocab: tamaño del vocabulario de salida.
        super(Generator, self).__init__()
        # Proyecta de d_model a la dimensión del vocabulario
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        # x: (Batch, Seq, d_model)
        # -> proj -> (Batch, Seq, vocab)
        # Aplicar log_softmax es común para usar con NLLLoss durante el entrenamiento
        return torch.log_softmax(self.proj(x), dim=-1)

class Transformer(nn.Module):
    # Implementación del modelo Transformer completo.
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed # Embedding + Positional Encoding para la entrada
        self.tgt_embed = tgt_embed # Embedding + Positional Encoding para la salida
        self.generator = generator # Capa lineal final + Softmax

    def forward(self, src, tgt, src_mask, tgt_mask):
        # Procesa secuencias de entrada y salida enmascaradas.
        # 1. Pasa la entrada por el codificador
        memory = self.encode(src, src_mask)
        # 2. Pasa la salida del codificador y la entrada del decodificador por el decodificador
        return self.decode(memory, src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        # Aplica embedding + pos encoding a la entrada, luego pasa por el codificador
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        # Aplica embedding + pos encoding a la entrada del decodificador, luego pasa por el decodificador
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

print("Clases Generator y Transformer definidas.")


Clases Generator y Transformer definidas.


###Función Constructora y Ejemplo de Uso
Creamos una función para construir el modelo con hiperparámetros específicos y probamos pasar datos ficticios a través de él.

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    # Construye un modelo Transformer completo con hiperparámetros.
    c = copy.deepcopy
    attn = MultiHeadAttention(h, d_model, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)

    # Crear el modelo completo
    model = Transformer(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )

    # Inicializar parámetros con Xavier Glorot (importante para convergencia)
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return model

# --- Ejemplo de uso --- #

# Parámetros de ejemplo (puedes cambiarlos)
SRC_VOCAB_SIZE = 1000 # Tamaño vocabulario fuente pequeño para ejemplo
TGT_VOCAB_SIZE = 1200 # Tamaño vocabulario objetivo pequeño para ejemplo
N_LAYERS = 2        # Número de capas (reducido para rapidez)
D_MODEL = 128       # Dimensión del modelo (reducido)
D_FF = 256          # Dimensión FeedForward (reducido)
NUM_HEADS = 4         # Número de cabezas (reducido)
DROPOUT = 0.1

# Crear modelo
model = make_model(SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, N=N_LAYERS, d_model=D_MODEL, d_ff=D_FF, h=NUM_HEADS, dropout=DROPOUT)
print(f"Modelo Transformer creado con {N_LAYERS} capas, d_model={D_MODEL}, {NUM_HEADS} cabezas.")
# print(model) # Descomenta para ver la estructura detallada

# Ejemplo de datos dummy (Batch=2, Seq_len_src=10, Seq_len_tgt=8)
# Tokens de entrada (valores entre 1 y SRC_VOCAB_SIZE-1, 0 es padding)
src_dummy = torch.randint(1, SRC_VOCAB_SIZE, (2, 10))
src_dummy[0, 7:] = 0 # Añadir padding a la primera secuencia
# Tokens de salida esperados (para entrenamiento teacher forcing)
# Valores entre 1 y TGT_VOCAB_SIZE-1, 0 es padding
tgt_dummy = torch.randint(1, TGT_VOCAB_SIZE, (2, 8))
tgt_dummy[1, 6:] = 0 # Añadir padding a la segunda secuencia
# La entrada real al decoder durante el entrenamiento es tgt excepto el último token
tgt_dummy_input = tgt_dummy[:, :-1]

# Crear máscaras
# Máscara de padding para src: (Batch, 1, Seq_len) - True donde NO es padding
src_mask_dummy = (src_dummy != 0).unsqueeze(1)
# Máscara de padding para tgt_input: (Batch, 1, Seq_len)
_tgt_mask_padding = (tgt_dummy_input != 0).unsqueeze(1)
# Máscara futura para tgt_input (triangular inferior): (1, Seq_len, Seq_len)
def subsequent_mask(size):
    "Enmascara posiciones futuras."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0 # Devuelve True donde NO está enmascarado

_tgt_mask_future = subsequent_mask(tgt_dummy_input.size(-1))
# Combinar máscara de padding y futura para el decoder
tgt_mask_dummy = _tgt_mask_padding & _tgt_mask_future

# Pasar datos por el modelo (solo forward pass, sin entrenamiento)
# Asegúrate de que el modelo esté en modo evaluación si no estás entrenando
model.eval()
with torch.no_grad(): # No calcular gradientes para este ejemplo
    output = model(src_dummy, tgt_dummy_input, src_mask_dummy, tgt_mask_dummy)
    # Pasar la salida por el generador para obtener log probabilidades
    final_output_log_probs = model.generator(output)

print(f"\nForma de la entrada src: {src_dummy.shape}")
print(f"Forma de la entrada tgt (input decoder): {tgt_dummy_input.shape}")
print(f"Forma de la máscara src: {src_mask_dummy.shape}")
print(f"Forma de la máscara tgt: {tgt_mask_dummy.shape}")
print(f"Forma de la salida del decoder (antes del generador): {output.shape}") # Esperado: (Batch, Seq_len_tgt-1, d_model)
print(f"Forma de la salida final (log probs): {final_output_log_probs.shape}") # Esperado: (Batch, Seq_len_tgt-1, tgt_vocab_size)

# Puedes inspeccionar los valores si quieres
# print("\nLog probabilidades del primer token de la primera secuencia:")
# print(final_output_log_probs[0, 0, :])


Modelo Transformer creado con 2 capas, d_model=128, 4 cabezas.

Forma de la entrada src: torch.Size([2, 10])
Forma de la entrada tgt (input decoder): torch.Size([2, 7])
Forma de la máscara src: torch.Size([2, 1, 10])
Forma de la máscara tgt: torch.Size([2, 7, 7])
Forma de la salida del decoder (antes del generador): torch.Size([2, 7, 128])
Forma de la salida final (log probs): torch.Size([2, 7, 1200])


**Explicación Práctica:**

*   Hemos definido todas las clases necesarias para un Transformer.
*   La función `make_model` facilita la creación del modelo con diferentes hiperparámetros.
*   El ejemplo de uso muestra cómo crear el modelo, preparar datos ficticios (incluyendo padding) y las máscaras correspondientes (padding y futura).
*   Finalmente, pasamos los datos por el modelo para obtener las log-probabilidades de salida. En un escenario real, estas se usarían con una función de pérdida (como `NLLLoss`) para entrenar el modelo.
*   **¡Experimenta!** Cambia los hiperparámetros (`N_LAYERS`, `D_MODEL`, `NUM_HEADS`), los tamaños de secuencia o batch y observa cómo afecta (aunque aquí solo hacemos un forward pass).

---


## Comparación Práctica: Transformers vs. RNNs/CNNs

Antes de los Transformers, las Redes Neuronales Recurrentes (RNNs, como LSTMs y GRUs) y las Redes Neuronales Convolucionales (CNNs) eran las arquitecturas dominantes en NLP.

*   **RNNs:** Procesan secuencias token por token, manteniendo un estado oculto que (idealmente) captura información de tokens anteriores. Son inherentemente secuenciales.
*   **CNNs:** Aplican filtros (convoluciones) sobre ventanas de tokens, capturando patrones locales. Se pueden apilar para capturar contextos más amplios, pero no son tan naturales para dependencias a largo plazo como las RNNs.

**Ventajas Prácticas de los Transformers:**

1.  **Paralelización Superior:** A diferencia del procesamiento secuencial de las RNNs, los cálculos dentro de un Transformer (especialmente la auto-atención y las capas feed-forward) pueden paralelizarse masivamente a lo largo de la dimensión de la secuencia. Esto permite entrenar modelos mucho más grandes en hardware moderno (GPUs/TPUs) de manera significativamente más rápida que RNNs equivalentes en tamaño.
2.  **Mejor Captura de Dependencias a Largo Plazo:** La auto-atención calcula directamente las interacciones entre cualquier par de tokens en la secuencia, sin importar su distancia. Esto supera la dificultad de las RNNs para propagar información a través de muchos pasos de tiempo (problema del desvanecimiento del gradiente), permitiendo a los Transformers modelar relaciones complejas en textos largos de manera más efectiva.
3.  **Representaciones Contextuales Ricas:** La auto-atención permite que la representación de cada token se base en una combinación ponderada de todos los demás tokens, generando embeddings contextuales muy potentes (ej. BERT).

**Desventajas Prácticas de los Transformers (Estándar):**

1.  **Complejidad Cuadrática con la Longitud de Secuencia:** La auto-atención estándar requiere calcular una puntuación para cada par de tokens, lo que resulta en una complejidad computacional y de memoria de O(n^2), donde n es la longitud de la secuencia. Esto hace que procesar secuencias muy largas (miles de tokens) sea computacionalmente muy costoso o inviable con la arquitectura original. Las RNNs tienen complejidad lineal O(n).
2.  **Mayor Necesidad de Datos y Cómputo (para modelos grandes):** Aunque pueden entrenarse más rápido debido a la paralelización, los modelos Transformer más potentes (como BERT, GPT) suelen ser muy grandes y requieren enormes cantidades de datos y recursos computacionales para su pre-entrenamiento.
3.  **Menos Inducción Secuencial Inherente:** No tienen un sesgo inductivo tan fuerte hacia el orden secuencial como las RNNs. La información posicional debe añadirse explícitamente (Codificación Posicional).

**En resumen:** Para la mayoría de las tareas de NLP modernas, las ventajas de paralelización y captura de contexto de los Transformers superan sus desventajas, especialmente con la disponibilidad de modelos pre-entrenados y arquitecturas eficientes para secuencias largas.

---

## 3. Atención Práctica: Queries, Keys y Values

El mecanismo central es la **Atención Escalada por Producto Punto (Scaled Dot-Product Attention)**. Funciona con tres vectores derivados de la entrada para cada token: Query (Q), Key (K) y Value (V).

*   **Query (Q):** Representa el token actual que está "preguntando" o buscando información relevante.
*   **Key (K):** Representa un token en la secuencia (potencialmente todos, incluida ella misma) que "anuncia" la información que posee.
*   **Value (V):** Representa el contenido o la información real del token asociado a la Key.

**Flujo de Cálculo:**

1.  **Scores:** Calcula qué tan relevante es cada Key para una Query dada. Se hace mediante el producto punto: `Score = Q * K^T`.
2.  **Escalado:** Divide los scores por la raíz cuadrada de la dimensión de los vectores Key (`d_k`) para estabilizar los gradientes: `Scaled_Score = Score / sqrt(d_k)`.
3.  **Máscara (Opcional):** Si se proporciona una máscara (ej. para ignorar padding o posiciones futuras), los scores correspondientes se establecen en un valor muy negativo (ej. -infinito) antes del softmax.
4.  **Ponderación (Softmax):** Aplica softmax a los scores escalados (y enmascarados) para obtener pesos de atención que suman 1. Estos pesos indican cuánta atención debe prestar la Query a cada Value: `Weights = softmax(Scaled_Score)`.
5.  **Salida:** Calcula la suma ponderada de los Values usando los pesos de atención: `Output = Weights * V`.


Ejemplo Numérico Simple

Veamos un ejemplo muy simplificado con un solo Query, tres Keys/Values y una dimensión `d_k` pequeña.

In [ ]:
import torch
import torch.nn.functional as F
import math

# Dimensiones simplificadas
d_k = 4 # Dimensión de Key/Query/Value
seq_len_k = 3 # Número de Keys/Values

# Vectores de ejemplo (Batch=1, Num_Heads=1)
# Suponemos que ya han pasado por las proyecciones lineales
query = torch.randn(1, 1, d_k) # (Batch, Seq_q=1, d_k)
keys = torch.randn(1, seq_len_k, d_k) # (Batch, Seq_k, d_k)
values = torch.randn(1, seq_len_k, d_k) # (Batch, Seq_k, d_k)

print(f"Query (Q) shape: {query.shape}")
print(f"Keys (K) shape: {keys.shape}")
print(f"Values (V) shape: {values.shape}")

# 1. Calcular Scores (Producto Punto)
# query: (1, 1, d_k) -> (1, d_k)
# keys.transpose(-2, -1): (1, d_k, Seq_k)
# scores: (1, 1, d_k) @ (1, d_k, Seq_k) -> (1, 1, Seq_k)
scores = torch.matmul(query, keys.transpose(-2, -1))
print(f"\nScores (Q @ K^T) shape: {scores.shape}")
print(f"Scores: {scores}")

# 2. Escalar
scaled_scores = scores / math.sqrt(d_k)
print(f"\nScaled Scores shape: {scaled_scores.shape}")
print(f"Scaled Scores: {scaled_scores}")

# 3. Máscara (Opcional) - Ejemplo: Ignorar el último Key/Value
# mask shape debe ser compatible para broadcast: (Batch, Seq_q, Seq_k)
mask = torch.tensor([[[True, True, False]]]) # (1, 1, 3) - True donde NO está enmascarado
print(f"\nMask: {mask}")
scaled_scores_masked = scaled_scores.masked_fill(mask == 0, -1e9)
print(f"Scaled Scores (Masked): {scaled_scores_masked}")

# 4. Ponderación (Softmax)
# Aplicamos softmax sobre la última dimensión (Seq_k)
attention_weights = F.softmax(scaled_scores_masked, dim=-1)
print(f"\nAttention Weights (Softmax) shape: {attention_weights.shape}")
print(f"Attention Weights: {attention_weights}")
# Nota: El peso para la posición enmascarada (la última) debería ser cercano a cero.

# 5. Salida (Suma Ponderada de Values)
# attention_weights: (1, 1, Seq_k)
# values: (1, Seq_k, d_k)
# output: (1, 1, Seq_k) @ (1, Seq_k, d_k) -> (1, 1, d_k)
output = torch.matmul(attention_weights, values)
print(f"\nOutput (Weighted Sum of V) shape: {output.shape}")
print(f"Output: {output}")


Query (Q) shape: torch.Size([1, 1, 4])
Keys (K) shape: torch.Size([1, 3, 4])
Values (V) shape: torch.Size([1, 3, 4])

Scores (Q @ K^T) shape: torch.Size([1, 1, 3])
Scores: tensor([[[-0.6446, -0.3494,  1.9690]]])

Scaled Scores shape: torch.Size([1, 1, 3])
Scaled Scores: tensor([[[-0.3223, -0.1747,  0.9845]]])

Mask: tensor([[[ True,  True, False]]])
Scaled Scores (Masked): tensor([[[-3.2232e-01, -1.7471e-01, -1.0000e+09]]])

Attention Weights (Softmax) shape: torch.Size([1, 1, 3])
Attention Weights: tensor([[[0.4632, 0.5368, 0.0000]]])

Output (Weighted Sum of V) shape: torch.Size([1, 1, 4])
Output: tensor([[[-0.6992, -0.0519,  0.6605,  1.1972]]])


**Explicación Práctica:**

*   El código sigue los 5 pasos descritos.
*   La `query` calcula su similitud (`scores`) con cada `key`.
*   Los `scores` se escalan y opcionalmente se enmascaran.
*   `softmax` convierte los scores en `attention_weights` (una distribución de probabilidad).
*   La `output` es una combinación de los `values`, ponderada por cuánta atención recibió cada uno. El `value` correspondiente a la `key` enmascarada contribuye muy poco o nada a la salida.
*   En la **Atención Multi-Cabeza**, este proceso se realiza en paralelo con diferentes proyecciones de Q, K, V (las "cabezas"), y los resultados se concatenan y proyectan linealmente al final.

---

## Modelos Preentrenados y Ajuste Fino (Fine-Tuning) con 🤗 Transformers

Entrenar un Transformer desde cero requiere muchos datos y recursos. En la práctica, es mucho más común usar **modelos pre-entrenados** y **ajustarlos (fine-tuning)** para una tarea específica. La librería `transformers` de Hugging Face 🤗 facilita enormemente este proceso.

**Conceptos Clave:**

*   **Pre-entrenamiento:** Modelos enormes (como BERT, GPT, T5) se entrenan en corpus masivos de texto no etiquetado (ej. Wikipedia, libros, web) con objetivos como predecir palabras enmascaradas (BERT) o predecir la siguiente palabra (GPT). Aprenden representaciones lingüísticas generales.
*   **Ajuste Fino (Fine-Tuning):** Se toma un modelo pre-entrenado y se continúa entrenando (generalmente solo las últimas capas o todas con una tasa de aprendizaje baja) en un conjunto de datos más pequeño y específico de la tarea (ej. clasificación de sentimientos, respuesta a preguntas).

**Modelos Populares:**

*   **BERT (Bidirectional Encoder Representations from Transformers):** Basado en el codificador. Excelente para tareas de comprensión (clasificación, NER, Q&A). Aprende contexto de izquierda a derecha y de derecha a izquierda.
*   **GPT (Generative Pre-trained Transformer):** Basado en el decodificador. Excelente para generación de texto (escritura creativa, chatbots). Es autorregresivo (predice el siguiente token basado en los anteriores).
*   **T5 (Text-to-Text Transfer Transformer):** Arquitectura Encoder-Decoder. Trata todas las tareas como "texto a texto" añadiendo prefijos (ej. "summarize: ...", "translate English to German: ..."). Muy versátil.
*   **DistilBERT:** Una versión más pequeña y rápida de BERT, creada mediante destilación del conocimiento. Ideal para entornos con recursos limitados.


### Instalación de Librerías

Necesitamos instalar `transformers`, `datasets` (para cargar/manejar datos fácilmente) y `evaluate` (para métricas).

In [ ]:
# ¡Asegúrate de ejecutar esta celda!
!pip install transformers datasets evaluate torch accelerate -q
!pip install numpy==1.23.5 -q


print("Librerías instaladas.")


Librerías instaladas.


Ejemplo Práctico: Fine-Tuning para Clasificación de Texto

Vamos a ajustar DistilBERT (una versión más ligera de BERT) para una tarea de clasificación de sentimientos usando un pequeño dataset de ejemplo.

**1. Cargar Dataset y Tokenizador**

Usaremos un dataset de ejemplo simple. En una aplicación real, usarías `datasets.load_dataset("nombre_del_dataset")`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np
import evaluate

# Modelo pre-entrenado a usar (ligero y rápido)
model_name = "distilbert-base-uncased"

# Cargar tokenizador asociado al modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Datos de ejemplo (muy pequeños para demostración)
texts = ["I love this movie, it's fantastic!",
         "This was the worst film I have ever seen.",
         "The acting was okay, but the plot was boring.",
         "Absolutely brilliant, a must-watch!",
         "I didn't like it very much.",
         "A masterpiece of cinema."]
labels = [1, 0, 0, 1, 0, 1] # 1: Positivo, 0: Negativo

# Crear un Dataset de Hugging Face
data = {"text": texts, "label": labels}
dataset = Dataset.from_dict(data)

# Dividir en entrenamiento y evaluación (simple split para ejemplo)
dataset = dataset.train_test_split(test_size=0.3) # 70% train, 30% test

print("Dataset de ejemplo:")
print(dataset)

# Función para tokenizar los textos
def tokenize_function(examples):
    # padding='max_length' asegura que todas las secuencias tengan la misma longitud
    # truncation=True corta secuencias más largas que max_length
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Aplicar tokenización a todo el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Eliminar la columna de texto original, ya no la necesitamos
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# Renombrar 'label' a 'labels' (esperado por el Trainer)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# Establecer el formato a tensores de PyTorch
tokenized_datasets.set_format("torch")

print("\nDataset tokenizado:")
print(tokenized_datasets)
print("\nEjemplo de entrada tokenizada:")
# Accessing the first element without slicing
print(tokenized_datasets["train"][0])




RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'dtypes'

**2. Cargar Modelo Pre-entrenado**

Cargamos DistilBERT pre-entrenado, pero especificamos que es para clasificación de secuencias y el número de etiquetas (2 en nuestro caso: positivo/negativo).


In [ ]:
# Cargar el modelo pre-entrenado para clasificación de secuencias
# num_labels indica cuántas clases de salida hay
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print(f"Modelo {model_name} cargado para clasificación con {model.config.num_labels} etiquetas.")


**3. Definir Métricas y Argumentos de Entrenamiento**

Necesitamos una función para calcular métricas durante la evaluación y definir los hiperparámetros para el `Trainer`.

In [ ]:
# Cargar métrica de evaluación (accuracy es común para clasificación)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Obtener predicciones tomando el índice con mayor logit
    predictions = np.argmax(logits, axis=-1)
    # Calcular la métrica
    return metric.compute(predictions=predictions, references=labels)

# Definir argumentos para el entrenamiento
training_args = TrainingArguments(
    output_dir="./results_classification", # Directorio donde guardar resultados/checkpoints
    evaluation_strategy="epoch",         # Evaluar al final de cada época
    num_train_epochs=3,                  # Número de épocas (pocas para ejemplo rápido)
    per_device_train_batch_size=2,       # Tamaño de batch pequeño para ejemplo
    per_device_eval_batch_size=2,
    warmup_steps=1,                      # Pasos de calentamiento (pocos para ejemplo)
    weight_decay=0.01,
    logging_dir="./logs_classification",   # Directorio para logs
    logging_steps=1,
    # load_best_model_at_end=True, # Opcional: cargar el mejor modelo al final
    # push_to_hub=False, # Opcional: subir a Hugging Face Hub
)

print("Argumentos de entrenamiento definidos.")


**4. Crear y Ejecutar el Trainer**

El `Trainer` de Hugging Face se encarga del bucle de entrenamiento y evaluación.

In [ ]:
# Crear el objeto Trainer
trainer = Trainer(
    model=model,                         # El modelo a entrenar
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=tokenized_datasets["train"], # Dataset de entrenamiento
    eval_dataset=tokenized_datasets["test"],  # Dataset de evaluación
    compute_metrics=compute_metrics,     # Función para calcular métricas
    tokenizer=tokenizer,                 # Tokenizador (útil para padding dinámico si no se hizo antes)
)

print("Trainer creado. Iniciando fine-tuning...")
# Iniciar el entrenamiento (ajuste fino)
trainer.train()

print("\nFine-tuning completado.")

# Evaluar el modelo final
eval_results = trainer.evaluate()
print("\nResultados de la evaluación final:")
print(eval_results)


**5. Guardar Modelo y Hacer Inferencia**

Una vez entrenado, puedes guardar tu modelo ajustado y usarlo para predecir en nuevos datos.

In [ ]:
# Guardar el modelo ajustado y el tokenizador
save_directory = "./fine_tuned_distilbert_classifier"
print(f"\nGuardando modelo en {save_directory}...")
trainer.save_model(save_directory)
# El tokenizador también se guarda automáticamente con save_model si se proporcionó al Trainer
# tokenizer.save_pretrained(save_directory) # Opcional si no se pasó al Trainer
print("Modelo guardado.")

# --- Cargar y usar para inferencia --- #
print("\nCargando modelo guardado para inferencia...")
# Cargar el modelo y tokenizador guardados
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_directory)

# Mover modelo a GPU si está disponible (importante para inferencia rápida)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval() # Poner en modo evaluación

# Texto nuevo para clasificar
new_text = "This movie was incredibly moving and well-acted."
print(f"\nClasificando nuevo texto: '{new_text}'")

# Tokenizar el nuevo texto
inputs = loaded_tokenizer(new_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
# Mover inputs a la misma GPU/CPU que el modelo
inputs = {k: v.to(device) for k, v in inputs.items()}

# Hacer la predicción
with torch.no_grad(): # No necesitamos gradientes para inferencia
    outputs = loaded_model(**inputs)
    logits = outputs.logits

# Obtener la clase predicha
predicted_class_id = torch.argmax(logits, dim=-1).item()

# Mapear ID a etiqueta (según nuestro dataset original)
predicted_label = "Positivo" if predicted_class_id == 1 else "Negativo"

print(f"Predicción: {predicted_label} (ID: {predicted_class_id})")

# Ejemplo con otro texto
new_text_2 = "A complete waste of time and money."
print(f"\nClasificando nuevo texto: '{new_text_2}'")
inputs_2 = loaded_tokenizer(new_text_2, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
with torch.no_grad():
    outputs_2 = loaded_model(**inputs_2)
    logits_2 = outputs_2.logits
predicted_class_id_2 = torch.argmax(logits_2, dim=-1).item()
predicted_label_2 = "Positivo" if predicted_class_id_2 == 1 else "Negativo"
print(f"Predicción: {predicted_label_2} (ID: {predicted_class_id_2})")


**Explicación Práctica:**

*   Hemos usado `transformers` y `datasets` para cargar un modelo pre-entrenado (DistilBERT), preparar un dataset simple y tokenizarlo.
*   Configuramos `TrainingArguments` y `Trainer` para manejar el bucle de fine-tuning y evaluación.
*   Entrenamos el modelo por unas pocas épocas en nuestros datos específicos.
*   Guardamos el modelo ajustado.
*   Finalmente, cargamos el modelo guardado y lo usamos para clasificar nuevos textos.
*   **¡Experimenta!** Prueba con otros modelos pre-entrenados (`bert-base-uncased`, `roberta-base`), cambia los hiperparámetros de entrenamiento, usa un dataset más grande de Hugging Face Hub (ej. `imdb`).

---


## 5. Limitaciones y Consideraciones Éticas (Prácticas)

Si bien los Transformers son potentes, es crucial ser consciente de sus limitaciones prácticas y las implicaciones éticas al usarlos:

**Limitaciones Prácticas:**

*   **Recursos Computacionales:** Entrenar modelos grandes desde cero o incluso ajustar modelos muy grandes requiere GPUs/TPUs potentes y tiempo considerable. La inferencia también puede ser costosa para modelos grandes.
*   **Longitud de Secuencia:** La complejidad cuadrática de la atención estándar limita la longitud de secuencia que se puede procesar eficientemente (aunque existen variantes como Longformer).
*   **"Alucinaciones" y Falta de Sentido Común:** Los modelos pueden generar texto que suena plausible pero es incorrecto factualmente o carece de sentido común. No "entienden" el mundo real, solo patrones estadísticos.
*   **Interpretabilidad:** Es difícil saber exactamente *por qué* un Transformer genera una salida específica, lo que es problemático en aplicaciones críticas.

**Consideraciones Éticas Prácticas:**

*   **Sesgo (Bias):** Los modelos aprenden sesgos (sociales, de género, raciales) presentes en los datos de entrenamiento masivos. Esto puede llevar a resultados injustos o discriminatorios. *Acción Práctica:* Audita tus modelos en busca de sesgos, usa datasets más equilibrados si es posible, y considera técnicas de mitigación de sesgos.
*   **Desinformación:** La capacidad de generar texto realista puede usarse para crear noticias falsas, spam, etc. *Acción Práctica:* Sé responsable con el uso de modelos generativos y considera implementar salvaguardas.
*   **Consumo Energético:** Entrenar modelos grandes tiene una huella de carbono significativa. *Acción Práctica:* Prefiere usar modelos pre-entrenados y ajustarlos, considera modelos más pequeños/eficientes (como DistilBERT) o técnicas de optimización (ver Sección 7) cuando sea posible.
*   **Privacidad:** Existe un riesgo (aunque bajo para modelos grandes) de que los modelos memoricen datos sensibles del entrenamiento. *Acción Práctica:* Asegúrate de que los datos de entrenamiento estén adecuadamente anonimizados, especialmente si son sensibles.

---


## 6. Aplicaciones en Big Data (Conceptual Práctico)

¿Cómo aplicaríamos Transformers a conjuntos de datos masivos (Big Data), como millones de registros médicos electrónicos (RME) para detectar eventos adversos a medicamentos?

**Pasos Prácticos Conceptuales:**

1.  **Infraestructura:** Necesitarás un entorno de computación distribuida (ej. clústeres en la nube como AWS SageMaker, Google AI Platform, Azure ML) con acceso a múltiples GPUs/TPUs.
2.  **Almacenamiento de Datos:** Los datos (RME de-identificados) probablemente residirán en un data lake o almacén de datos distribuido (ej. S3, GCS, HDFS).
3.  **Preprocesamiento Distribuido:** Usar frameworks como **Apache Spark** (con PySpark) o **Dask** para:
    *   Leer datos en paralelo desde el almacenamiento distribuido.
    *   Realizar limpieza, de-identificación (¡crucial!), y tokenización distribuida (posiblemente usando `datasets.map` con procesamiento distribuido si se integra con Spark/Dask, o UDFs de Spark con el tokenizador).
    *   Guardar los datos preprocesados/tokenizados de nuevo en formato eficiente (ej. Parquet).
4.  **Entrenamiento Distribuido:** Utilizar librerías que se integren con PyTorch para paralelizar el entrenamiento del Transformer (ej. ajuste fino de ClinicalBERT) en múltiples GPUs/nodos:
    *   **PyTorch DistributedDataParallel (DDP):** Estándar de PyTorch para paralelismo de datos.
    *   **Hugging Face Accelerate:** Simplifica el uso de DDP y otras estrategias (como DeepSpeed) con los modelos y `Trainer` de Transformers.
    *   **Horovod:** Otro framework popular para entrenamiento distribuido.
    *   **DeepSpeed:** Librería de Microsoft para entrenar modelos masivos, optimizando memoria y velocidad (paralelismo de datos, tensor, pipeline, optimizador ZeRO).
    *   **Ejemplo con Accelerate/Trainer:** El `Trainer` de Hugging Face se integra con `accelerate`. Simplemente lanzando tu script de entrenamiento con `accelerate launch tu_script.py --args...` en un entorno configurado, `accelerate` maneja la distribución.
5.  **Inferencia Distribuida:** Para aplicar el modelo entrenado a nuevos datos a gran escala, también se pueden usar frameworks distribuidos (Spark UDFs con el modelo cargado, Dask) o servicios de inferencia optimizados (ej. NVIDIA Triton Inference Server, SageMaker Endpoints).
6.  **Monitorización y Gestión:** Usar herramientas para monitorizar el uso de recursos, el progreso del entrenamiento y gestionar los experimentos (ej. MLflow, Weights & Biases).

**Consideraciones de Código:**

*   **Manejo de Memoria:** Cargar modelos grandes y batches de datos requiere GPUs con mucha VRAM. Técnicas como gradient accumulation (en `TrainingArguments`), precisión mixta (`fp16=True`), y DeepSpeed (ZeRO) son esenciales.
*   **Eficiencia I/O:** Leer datos eficientemente desde el almacenamiento distribuido es clave. Formatos como Parquet o TFRecord son preferibles a archivos de texto plano.
*   **Robustez:** El código debe manejar fallos transitorios en el clúster (ej. reintentos, checkpoints).


In [ ]:
# Ejemplo conceptual (No ejecutable directamente aquí)
# Asume que tienes un script 'train_ehr.py' que usa Hugging Face Trainer

# Configurar accelerate (una vez por nodo/máquina)
# !accelerate config

# Lanzar entrenamiento distribuido (ej. en 4 GPUs)
# !accelerate launch --num_processes=4 train_ehr.py \
#   --output_dir ./ehr_output \
#   --model_name_or_path emilyalsentzer/Bio_ClinicalBERT \
#   --train_file path/to/distributed/train.parquet \
#   --validation_file path/to/distributed/eval.parquet \
#   --do_train --do_eval \
#   --num_train_epochs 1 \
#   --per_device_train_batch_size 8 \
#   --fp16 # Usar precisión mixta

print("Conceptual: Lanzamiento de entrenamiento distribuido con Accelerate.")


**Desafíos Clave:** Privacidad (HIPAA/GDPR), heterogeneidad de datos médicos, coste computacional, interpretabilidad.

---


## 7. Optimización y Eficiencia (Práctica Conceptual)

Los modelos Transformer grandes pueden ser lentos y consumir mucha memoria. Aquí hay un vistazo práctico a cómo optimizarlos:

**Técnicas Comunes:**

*   **Cuantización (Quantization):** Reduce la precisión de los números (pesos/activaciones) de 32 bits (FP32) a 16 bits (FP16/BF16) o 8 bits (INT8). Esto reduce el tamaño del modelo (~4x para INT8) y acelera la inferencia en hardware compatible (~2-4x).
*   **Poda (Pruning):** Elimina pesos o estructuras menos importantes del modelo. Puede reducir significativamente el tamaño y los cálculos, a veces con poca pérdida de precisión.
*   **Destilación del Conocimiento (Knowledge Distillation):** Entrena un modelo más pequeño ("estudiante") para imitar a uno más grande ("profesor"). El estudiante aprende de las predicciones del profesor, logrando un buen rendimiento con menos tamaño/velocidad (ej. DistilBERT).
*   **Arquitecturas Eficientes:** Usar variantes de Transformer diseñadas para ser más rápidas o manejar secuencias largas (ej. Longformer, Linformer, Reformer).

### 7.1. Ejemplo Práctico: Cuantización con PyTorch

PyTorch ofrece herramientas para cuantización post-entrenamiento (PTQ) y entrenamiento consciente de la cuantización (QAT).

In [ ]:
import torch
import torch.quantization
import copy

# --- Cuantización Dinámica Post-Entrenamiento (PTQ) ---
# Más simple, buena para LSTMs/Transformers, cuantiza solo pesos

# Carga tu modelo FP32 entrenado (usaremos una red lineal simple como ejemplo)
# En la práctica, cargarías tu modelo Transformer ajustado
fp32_model = nn.Sequential(
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
fp32_model.eval() # Poner en modo evaluación

# Aplicar cuantización dinámica (para capas Lineales y RNNs)
# backend=\'qnnpack\' es común para ARM, \'fbgemm\' para x86
quantized_dynamic_model = torch.quantization.quantize_dynamic(
    fp32_model, {nn.Linear}, dtype=torch.qint8, inplace=False
)

print("Modelo original (FP32):")
# print(fp32_model)
print("\nModelo cuantizado dinámicamente (INT8 pesos para Lineales):")
# print(quantized_dynamic_model)

# Comparar tamaño (conceptual)
# import os
# torch.save(fp32_model.state_dict(), "fp32_model.pth")
# torch.save(quantized_dynamic_model.state_dict(), "quantized_dynamic_model.pth")
# print(f"Tamaño FP32: {os.path.getsize(\'fp32_model.pth\') / 1e6:.2f} MB")
# print(f"Tamaño INT8 Dinámico: {os.path.getsize(\'quantized_dynamic_model.pth\') / 1e6:.2f} MB") # Debería ser ~1/4 para pesos lineales

# --- Cuantización Estática Post-Entrenamiento (PTQ) ---
# Requiere calibración con datos, cuantiza pesos y activaciones

# Clona el modelo original
static_model = copy.deepcopy(fp32_model)
static_model.eval()

# Especificar configuración de cuantización
static_model.qconfig = torch.quantization.get_default_qconfig(\'fbgemm\') # o \'qnnpack\'

# Fusionar módulos (Conv-BN-ReLU, etc.) si aplica (no en este ejemplo simple)
# static_model_fused = torch.quantization.fuse_modules(static_model, [['0', '1']], inplace=False)

# Preparar el modelo para la calibración
static_model_prepared = torch.quantization.prepare(static_model, inplace=False)

# Calibrar con datos representativos (ej. algunos batches del dataset de validación)
# print("\nCalibrando modelo estático...")
# with torch.no_grad():
#     for i in range(10): # Usar datos reales aquí
#         dummy_input = torch.randn(1, 128)
#         static_model_prepared(dummy_input)

# Convertir a modelo cuantizado
# static_model_quantized = torch.quantization.convert(static_model_prepared, inplace=False)
# print("Modelo cuantizado estáticamente (INT8 pesos y activaciones):")
# print(static_model_quantized)

# --- Entrenamiento Consciente de Cuantización (QAT) ---
# Mejor precisión, simula cuantización durante el entrenamiento

# qat_model = copy.deepcopy(fp32_model)
# qat_model.train() # Poner en modo entrenamiento
# qat_model.qconfig = torch.quantization.get_default_qat_qconfig(\'fbgemm\')
# qat_model_fused = torch.quantization.fuse_modules(qat_model, [['0', '1']], inplace=False)
# qat_model_prepared = torch.quantization.prepare_qat(qat_model_fused, inplace=False)

# Entrenar el modelo preparado (qat_model_prepared) por algunas épocas...
# print("\n(Conceptual) Entrenando con QAT...")

# Después del entrenamiento, convertir a modelo cuantizado
# qat_model_prepared.eval()
# qat_model_quantized = torch.quantization.convert(qat_model_prepared, inplace=False)
# print("(Conceptual) Modelo final cuantizado con QAT:")
# print(qat_model_quantized)

print("\nEjemplos conceptuales de cuantización con PyTorch.")


### 7.2. Hugging Face Optimum

La librería `optimum` de Hugging Face simplifica la aplicación de técnicas de optimización (cuantización, ONNX Runtime) a los modelos de `transformers`.

In [ ]:
# !pip install optimum[onnxruntime]

from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# Directorio del modelo ajustado de la sección anterior
model_checkpoint_dir = "./fine_tuned_distilbert_classifier"
# Directorio de salida para el modelo ONNX cuantizado
onnx_quantized_output_dir = "./onnx_quantized_distilbert"

# --- Cuantización Post-Entrenamiento con Optimum --- #

# 1. Crear cuantizador desde el modelo ajustado
ort_quantizer = ORTQuantizer.from_pretrained(model_checkpoint_dir)

# 2. Definir configuración de cuantización (ej. AVX2 para CPU, INT8)
# qconfig = AutoQuantizationConfig.avx2(config=None, is_static=False) # Dinámica
qconfig = AutoQuantizationConfig.avx2(config=None, is_static=True) # Estática (requiere dataset de calibración)

# 3. (Solo para estática) Cargar dataset de calibración
# from datasets import load_dataset
# calibration_dataset = ort_quantizer.get_calibration_dataset(
#     "glue", # Ejemplo: dataset GLUE
#     dataset_config_name="sst2", # Ejemplo: tarea SST-2
#     preprocess_function=lambda examples: tokenizer(examples["sentence"], padding="max_length", truncation=True),
#     num_samples=50, # Número de muestras para calibrar
#     dataset_split="train",
# )

# 4. Cuantizar a formato ONNX
# Para estática, añadir: calibration_dataset=calibration_dataset
ort_quantizer.quantize(
    save_dir=onnx_quantized_output_dir,
    quantization_config=qconfig,
)

print(f"Modelo cuantizado y exportado a ONNX en {onnx_quantized_output_dir}")

# --- Cargar y usar modelo ONNX cuantizado --- #
# print("\nCargando modelo ONNX cuantizado...")
# loaded_onnx_model = ORTModelForSequenceClassification.from_pretrained(onnx_quantized_output_dir)
# loaded_onnx_tokenizer = AutoTokenizer.from_pretrained(onnx_quantized_output_dir)

# new_text = "This is a great tool!"
# inputs = loaded_onnx_tokenizer(new_text, return_tensors="pt")

# outputs = loaded_onnx_model(**inputs)
# logits = outputs.logits
# predicted_class_id = torch.argmax(logits, dim=-1).item()
# print(f"Predicción ONNX: {"Positivo" if predicted_class_id == 1 else "Negativo"}")




**Explicación Práctica:**

*   PyTorch ofrece APIs para cuantización dinámica, estática y QAT. La dinámica es la más simple, la estática requiere calibración, y QAT da la mejor precisión pero requiere reentrenamiento.
*   Hugging Face `optimum` simplifica la cuantización (especialmente a formato ONNX para inferencia acelerada con ONNX Runtime) de modelos `transformers`.
*   La elección depende del hardware objetivo (CPU/GPU/TPU), los requisitos de precisión y la complejidad de implementación aceptable.

---

## 8. Diseño Personalizado: Resumen Abstractivo con 🤗 Transformers

Diseñar un modelo "personalizado" a menudo significa adaptar una arquitectura pre-entrenada para una tarea específica. Vamos a usar un modelo pre-entrenado Encoder-Decoder (como BART o PEGASUS) para **resumen abstractivo** (generar un resumen que no solo copia frases).

**Tarea:** Dado un texto largo (ej. artículo), generar un resumen corto.

**Modelo:** Usaremos `google/pegasus-xsum`, un modelo pre-entrenado específicamente para resúmenes cortos y abstractivos (entrenado en el dataset XSum).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Cargar modelo y tokenizador pre-entrenado para resumen
model_name_summarization = "google/pegasus-xsum"
print(f"Cargando modelo {model_name_summarization}...")

# Usar try-except por si hay problemas de conexión o memoria en Colab
try:
    tokenizer_summarization = AutoTokenizer.from_pretrained(model_name_summarization)
    model_summarization = AutoModelForSeq2SeqLM.from_pretrained(model_name_summarization)

    # Mover a GPU si está disponible
    device_sum = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_summarization.to(device_sum)
    model_summarization.eval() # Modo evaluación

    print("Modelo de resumen cargado.")

    # Texto de ejemplo para resumir (puedes reemplazarlo)
    ARTICLE_TO_SUMMARIZE = ("""
    Scientists have discovered a new species of deep-sea fish living near hydrothermal vents
    in the Pacific Ocean. The fish, temporarily named Ventichthys чудо (chudo, meaning wonder),
    possesses unique bioluminescent features never before seen. Adapted to extreme pressure
    and temperature, its discovery challenges existing theories about the limits of life
    on Earth. Researchers used a remotely operated vehicle (ROV) to capture footage and
    samples at depths exceeding 3,000 meters. Further genetic analysis is underway to
    understand its evolutionary lineage and unique adaptations. The finding highlights
    how much of the deep ocean remains unexplored and the potential for discovering
    entirely new ecosystems and life forms.
    """)

    print(f"\nTexto a resumir:\n{ARTICLE_TO_SUMMARIZE}")

    # Tokenizar el texto de entrada
    inputs_summarization = tokenizer_summarization(ARTICLE_TO_SUMMARIZE,
                                                   max_length=1024, # Longitud máxima de entrada para PEGASUS
                                                   return_tensors="pt",
                                                   truncation=True).to(device_sum)

    # Generar el resumen
    # Parámetros de generación comunes:
    # - max_length: Longitud máxima del resumen generado
    # - min_length: Longitud mínima del resumen generado
    # - num_beams: Usa beam search para mejor calidad (más lento)
    # - length_penalty: Penaliza resúmenes más largos/cortos
    # - no_repeat_ngram_size: Evita repetir n-gramas
    print("\nGenerando resumen...")
    with torch.no_grad():
        summary_ids = model_summarization.generate(inputs_summarization["input_ids"],
                                                 num_beams=4,
                                                 min_length=30,
                                                 max_length=60, # Resumen corto para XSum
                                                 early_stopping=True)

    # Decodificar los IDs generados a texto
    generated_summary = tokenizer_summarization.decode(summary_ids[0], skip_special_tokens=True)

    print(f"\nResumen Generado:\n{generated_summary}")

except Exception as e:
    print(f"Error al cargar o usar el modelo de resumen: {e}")
    print("Esto puede deberse a memoria insuficiente en Colab o problemas de red.")
    print("Intenta reiniciar el entorno de ejecución o usar un modelo más pequeño si es necesario.")




**Explicación Práctica:**

*   Cargamos un modelo Seq2Seq (`AutoModelForSeq2SeqLM`) y su tokenizador, específicamente `pegasus-xsum` que está optimizado para resúmenes cortos.
*   Tokenizamos el artículo de entrada.
*   Usamos el método `model.generate()` para producir el resumen. Este método implementa la decodificación (a menudo usando beam search) para generar la secuencia de salida token por token.
*   Ajustamos los parámetros de `generate` (como `num_beams`, `max_length`, `min_length`) para controlar la calidad y longitud del resumen.
*   Finalmente, decodificamos los IDs de token generados para obtener el resumen en texto.
*   **¡Experimenta!** Prueba con diferentes textos, ajusta los parámetros de `generate`, o incluso intenta con otro modelo pre-entrenado para resumen (ej. `google/bart-large-cnn`).

---

## Conclusión y Próximos Pasos

Este cuaderno ha proporcionado una introducción práctica a los Transformers con PyTorch, cubriendo desde la implementación básica hasta el uso avanzado con modelos pre-entrenados y técnicas de optimización.

**Puntos Clave:**

*   Los Transformers se basan en la **atención** para capturar relaciones en los datos.
*   Son altamente **paralelizables**, permitiendo modelos muy grandes.
*   El **pre-entrenamiento y ajuste fino** con librerías como 🤗 Transformers es el enfoque estándar.
*   Existen **limitaciones** (recursos, longitud de secuencia) y **consideraciones éticas** (sesgo, desinformación) importantes.
*   Las técnicas de **optimización** (cuantización, poda) y las **arquitecturas eficientes** ayudan a desplegar modelos en entornos prácticos.

**Próximos Pasos:**

*   Profundiza en tareas específicas (Q&A, NER, Traducción) usando `transformers`.
*   Explora diferentes modelos pre-entrenados en [Hugging Face Hub](https://huggingface.co/models).
*   Aprende más sobre técnicas de optimización con `optimum` o PyTorch.
*   Intenta entrenar un modelo desde cero (si tienes suficientes datos y recursos) o ajustar modelos más grandes.
*   Contribuye a la comunidad de código abierto.

¡Esperamos que esta guía práctica te haya sido útil en tu viaje de aprendizaje sobre Transformers!

---

## Referencias (Simplificado)

*   Vaswani et al. (2017). Attention is all you need. [Link](https://arxiv.org/abs/1706.03762)
*   Devlin et al. (2018). BERT: Pre-training of Deep Bidirectional Transformers... [Link](https://arxiv.org/abs/1810.04805)
*   Radford et al. (GPT series). OpenAI Blog.
*   Raffel et al. (2020). Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. [Link](https://arxiv.org/abs/1910.10683)
*   Wolf et al. (2019). HuggingFace's Transformers: State-of-the-Art Natural Language Processing. [Link](https://arxiv.org/abs/1910.03771)
*   Zhang et al. (2020). PEGASUS: Pre-training with Extracted Gap-sentences... [Link](https://arxiv.org/abs/1912.08777)
*   PyTorch Documentation: [Link](https://pytorch.org/docs/stable/index.html)
*   Hugging Face Documentation (Transformers, Datasets, Evaluate, Optimum): [Link](https://huggingface.co/docs)

---



## Otra práctica

La estructura base del Transformer sencillo en PyTorch

In [5]:
# Importamos librerias
import torch
import torch.nn as nn
import torch.nn.functional as F

# Clase MultiHeadAttention: Implementa la atención multi-cabeza
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        # Verificamos que d_model sea divisible por el número de cabezas
        assert d_model % n_heads == 0
        self.d_k = d_model // n_heads  # Dimensión de cada cabeza
        self.n_heads = n_heads

        # Definimos capas lineales para Q, K, V y la salida
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def attention(self, Q, K, V):
        # Calculamos los scores de atención escalados
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float32))
        # Aplicamos softmax para obtener los pesos de atención
        attn_weights = F.softmax(scores, dim=-1)
        # Calculamos la salida ponderada
        output = torch.matmul(attn_weights, V)
        return output, attn_weights

    def forward(self, x):
        batch_size = x.size(0)

        # Aplicamos las capas lineales para obtener Q, K, V
        Q = self.q_linear(x)
        K = self.k_linear(x)
        V = self.v_linear(x)

        # Redimensionamos para aplicar atención multi-cabeza
        Q = Q.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        # Llamamos a la función de atención
        attn_output, _ = self.attention(Q, K, V)

        # Volvemos a unir las cabezas de atención
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_k * self.n_heads)
        # Aplicamos la capa de salida
        output = self.out_linear(attn_output)
        return output

# Clase FeedForward: Implementa el MLP del Transformer
class FeedForward(nn.Module):
    def __init__(self, d_model, ff_dim, dropout=0.1):
        super(FeedForward, self).__init__()
        # Capa intermedia con ReLU
        self.fc1 = nn.Linear(d_model, ff_dim)
        # Capa de salida
        self.fc2 = nn.Linear(ff_dim, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Aplicamos ReLU y Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # Capa de salida
        x = self.fc2(x)
        return x

# Bloque Transformer: Implementa una capa de Transformer
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        # Módulo de atención multi-cabeza
        self.attention = MultiHeadAttention(d_model, n_heads)
        # Normalización de la salida de la atención
        self.norm1 = nn.LayerNorm(d_model)
        # Módulo feedforward
        self.ff = FeedForward(d_model, ff_dim, dropout)
        # Normalización de la salida del feedforward
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Aplicamos atención y normalización
        attn_output = self.attention(x)
        x = self.norm1(x + self.dropout(attn_output))
        # Aplicamos feedforward y normalización
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

# Modelo Transformer Simple
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, d_model, n_heads, ff_dim, n_layers, dropout=0.1):
        super(SimpleTransformer, self).__init__()
        # Capa de embedding inicial
        self.embedding = nn.Linear(input_dim, d_model)
        # Múltiples capas Transformer
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)
        ])
        # Capa de salida
        self.fc_out = nn.Linear(d_model, input_dim)

    def forward(self, x):
        # Aplicamos el embedding inicial
        x = self.embedding(x)
        # Pasamos por cada capa Transformer
        for layer in self.layers:
            x = layer(x)
        # Capa de salida final
        output = self.fc_out(x)
        return output

# Definición de hiperparámetros
input_dim = 10  # Dimensión de entrada

# Parámetros del modelo
d_model = 128
n_heads = 4
ff_dim = 512
n_layers = 2

# Inicialización del modelo
model = SimpleTransformer(input_dim, d_model, n_heads, ff_dim, n_layers)
print(model)

SimpleTransformer(
  (embedding): Linear(in_features=10, out_features=128, bias=True)
  (layers): ModuleList(
    (0-1): 2 x TransformerBlock(
      (attention): MultiHeadAttention(
        (q_linear): Linear(in_features=128, out_features=128, bias=True)
        (k_linear): Linear(in_features=128, out_features=128, bias=True)
        (v_linear): Linear(in_features=128, out_features=128, bias=True)
        (out_linear): Linear(in_features=128, out_features=128, bias=True)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (fc_out): Linear(in_features=128, out_features=10, bias=True)
)


Traductor de idiomas

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Clase AttentionMechanism: Implementa un mecanismo de atención simple
class AttentionMechanism(nn.Module):
    def __init__(self, d_model):
        super(AttentionMechanism, self).__init__()
        # Capas lineales para Query, Key y Value
        self.query_layer = nn.Linear(d_model, d_model)
        self.key_layer = nn.Linear(d_model, d_model)
        self.value_layer = nn.Linear(d_model, d_model)

    def forward(self, query, key, value):
        # Aplicamos capas lineales para obtener Q, K, V
        Q = self.query_layer(query)  # Transformamos el Query
        K = self.key_layer(key)      # Transformamos el Key
        V = self.value_layer(value)  # Transformamos el Value

        # Calculamos los scores de atención
        # Multiplicamos Q por la transpuesta de K y escalamos por la raíz cuadrada de la dimensión
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(Q.size(-1), dtype=torch.float32))

        # Aplicamos softmax para convertir los scores en probabilidades de atención
        attn_weights = F.softmax(scores, dim=-1)

        # Multiplicamos los pesos de atención por los valores V
        output = torch.matmul(attn_weights, V)

        return output, attn_weights

# Ejemplo práctico: Traducción de inglés a español
# Supongamos que tenemos un diccionario simple con palabras embebidas
embedding_dim = 16  # Dimensión del embedding

# Secuencia en inglés (5 palabras representadas como vectores de 16 dimensiones)
english_sentence = torch.randn((1, 5, embedding_dim))  # (batch_size, seq_len, embedding_dim)

# Secuencia en español (7 palabras representadas como vectores de 16 dimensiones)
spanish_sentence = torch.randn((1, 7, embedding_dim))  # (batch_size, seq_len, embedding_dim)

# Inicializamos el mecanismo de atención
attention = AttentionMechanism(embedding_dim)

# Aplicamos atención utilizando la oración en inglés como Query y la oración en español como Key/Value
output, attn_weights = attention(english_sentence, spanish_sentence, spanish_sentence)

# Salida del mecanismo de atención
print("Output Shape (context vector):", output.shape)  # (batch_size, seq_len_english, embedding_dim)
print("Attention Weights Shape:", attn_weights.shape)  # (batch_size, seq_len_english, seq_len_spanish)

Output Shape (context vector): torch.Size([1, 5, 16])
Attention Weights Shape: torch.Size([1, 5, 7])


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer

# Clase TransformerEncoder: Implementa el Encoder del modelo Transformer
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        # Módulo de Auto-Atención Multi-Cabeza
        self.self_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_heads, dropout=dropout)
        # Capa Feed Forward
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim),  # Proyección a mayor dimensión
            nn.ReLU(),  # Función de activación no lineal
            nn.Dropout(dropout),  # Regularización
            nn.Linear(ff_dim, d_model)  # Proyección de vuelta a la dimensión original
        )
        # Capas de Normalización
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        # Paso 1: Auto-atención sobre la entrada
        attn_output, _ = self.self_attention(x, x, x)
        # Residual + Normalización
        x = self.norm1(x + attn_output)
        # Paso 2: Feedforward
        ff_output = self.ff(x)
        # Residual + Normalización
        x = self.norm2(x + ff_output)
        return x

# Clase TransformerDecoder: Implementa el Decoder del Transformer
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        # Auto-Atención del Decoder
        self.self_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_heads, dropout=dropout)
        # Atención Cruzada con la salida del Encoder
        self.cross_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=n_heads, dropout=dropout)
        # Capa Feed Forward
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, d_model)
        )
        # Capas de Normalización
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, enc_output):
        # Auto-atención del Decoder
        attn_output, _ = self.self_attention(x, x, x)
        x = self.norm1(x + attn_output)
        # Atención Cruzada con la salida del Encoder
        cross_output, _ = self.cross_attention(x, enc_output, enc_output)
        x = self.norm2(x + cross_output)
        # Feedforward
        ff_output = self.ff(x)
        x = self.norm3(x + ff_output)
        return x

# Clase TransformerSummarizer: Implementa el modelo completo para resumen
class TransformerSummarizer(nn.Module):
    def __init__(self, input_dim, d_model, n_heads, ff_dim, n_layers, dropout=0.1):
        super(TransformerSummarizer, self).__init__()
        # Capa de Embedding para convertir tokens en vectores
        self.embedding = nn.Embedding(input_dim, d_model)
        # Lista de capas Encoder
        self.encoder_layers = nn.ModuleList([
            TransformerEncoder(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)
        ])
        # Lista de capas Decoder
        self.decoder_layers = nn.ModuleList([
            TransformerDecoder(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)
        ])
        # Capa de salida para generar las palabras del resumen
        self.output_layer = nn.Linear(d_model, input_dim)

    def forward(self, src, tgt):
        # Embedding de entrada (texto original)
        src = self.embedding(src)
        # Embedding de la secuencia objetivo (resumen)
        tgt = self.embedding(tgt)
        # Paso por las capas del Encoder
        for layer in self.encoder_layers:
            src = layer(src)
        # Paso por las capas del Decoder
        for layer in self.decoder_layers:
            tgt = layer(tgt, src)
        # Generación de la secuencia de salida
        output = self.output_layer(tgt)
        return output

# Definición de hiperparámetros
input_dim = 30522  # Tamaño del vocabulario BERT (ejemplo)
d_model = 256
n_heads = 8
ff_dim = 512
n_layers = 4

# Inicialización del modelo con los hiperparámetros definidos
model = TransformerSummarizer(input_dim, d_model, n_heads, ff_dim, n_layers)
print(model)  # Visualización de la estructura del modelo


TransformerSummarizer(
  (embedding): Embedding(30522, 256)
  (encoder_layers): ModuleList(
    (0-3): 4 x TransformerEncoder(
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (ff): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
        (3): Linear(in_features=512, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (decoder_layers): ModuleList(
    (0-3): 4 x TransformerDecoder(
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (cross_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256,